In [1]:
import pandas as pd
#df = pd.read_excel('time_series.xlsx', sheet_name='60min')

In [2]:
df = pd.read_csv('data/time_series_60min_stacked.csv')

In [3]:
#Which vairiable + attribute combos have the most data
#df.groupby(['variable','attribute']).count().sort_values(by='data', ascending=False)

In [20]:
subset_columns = df[(df['variable'].isin(['wind','solar'])) & (df['attribute'].isin(['capacity']))]

In [69]:
wind = df[(df['variable'].isin(['wind_onshore'])) & (df['attribute'].isin(['generation_actual']))]
solar = df[(df['variable'].isin(['solar'])) & (df['attribute'].isin(['generation_actual']))]

In [70]:
#which countries have the most data?
#subset_columns.groupby('region').count().sort_values(by='data',ascending=False)

In [48]:
solar.region.unique().shape

(39,)

In [72]:
wind.region.unique().shape

(59,)

In [77]:
wind_countries = wind#[wind['region'].isin(['DK','DE','SE','GB','IT','AT','CZ','FR','BR','PT','ES','EE','NL'])]
solar_countries = solar#[solar['region'].isin(['DK','DE','SE','GB','IT','AT','CZ','FR','BR','PT','ES','EE','NL'])]

In [78]:
wind_countries['region_variable_attribute'] = wind_countries['region']+wind_countries['variable']+wind_countries['attribute']
solar_countries['region_variable_attribute'] = solar_countries['region']+solar_countries['variable']+solar_countries['attribute']


/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
wind_countries['timestamp'] = pd.to_datetime(wind_countries['utc_timestamp'], format='%Y-%m-%dT%H:%M:%SZ')
solar_countries['timestamp'] = pd.to_datetime(solar_countries['utc_timestamp'], format='%Y-%m-%dT%H:%M:%SZ')

/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Make data a float

In [163]:
wind_countries['data']=wind_countries['data'].astype('float')

/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Pivot the tables

In [164]:
wind_pivot=wind_countries.pivot(index='timestamp', columns='region', values='data')#.fillna(0)
solar_pivot=solar_countries.pivot(index='timestamp', columns='region', values='data')#.fillna(0)

When do most countries start recording their data?

In [165]:
# mean earliest record start time. Exclude the years before this date.
wind_start_time = wind_countries.groupby('region').min()['timestamp'].mean()

In [166]:
# mean earliest record start time. Exclude the years before this date. About the same as wind
solar_start_time = solar_countries.groupby('region').min()['timestamp'].mean()

In [167]:
# number of rows per year relative to the highest recorded year
# supports the dates above more or less. Perhaps it'd also be good to start the record at 2015

solar_pivot.groupby(solar_pivot.index.year).count().sum(axis=1)

timestamp
2010     24168
2011     43792
2012     83053
2013     88208
2014    107474
2015    331112
2016    324245
2017    323842
2018    323419
2019     92806
dtype: int64

Take a subset of the time

In [238]:
wind_subset_time = wind_pivot[wind_pivot.index >= wind_start_time]
solar_subset_time = wind_pivot[wind_pivot.index >= solar_start_time]

In [239]:
# not enough rows if we group by day. Grouping by hours, which is redundant
'''
wind_final = wind_subset_time.groupby(pd.Grouper(freq='h')).mean()
solar_final = solar_subset_time.groupby(pd.Grouper(freq='h')).mean()
'''

"\nwind_final = wind_subset_time.groupby(pd.Grouper(freq='h')).mean()\nsolar_final = solar_subset_time.groupby(pd.Grouper(freq='h')).mean()\n"

#### Solar

Summing and dropping the country regions where necessary

- Dropping bidding and control regions of: IT, DE, DK, IE (includes northern Ireland which is UK not IE), NO, SE
- Summing and dropping regions: GB
  - sum['GB_GBN', 'GB_NIR', 'GB_UKM'] --> 'GB'

In [250]:
#solar_subset_time.isnull().sum(axis=0)

In [241]:
#select column names excluding the country name, only control and bidding zones
DE_cols = [col for col in solar_subset_time.columns if 'DE' in col][1:]
DK_cols = [col for col in solar_subset_time.columns if 'DK' in col][1:]
IT_cols = [col for col in solar_subset_time.columns if 'IT' in col][1:]
GB_cols = [col for col in solar_subset_time.columns if 'GB' in col]
IE_cols = [col for col in solar_subset_time.columns if 'IE' in col][1:]
NO_cols = [col for col in solar_subset_time.columns if 'NO' in col][1:]
SE_cols = [col for col in solar_subset_time.columns if 'SE' in col][1:]

In [242]:
# new GB column
solar_subset_time['GB'] = solar_subset_time[GB_cols].sum(axis=1)

/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [289]:
solar_semi_final = solar_subset_time.drop(columns=DE_cols+DK_cols+IT_cols+GB_cols+IE_cols+NO_cols+SE_cols)

Drop countries that still have too many null values by finding a threshold

In [290]:
# Require that each country have this many non-NA values.
# threshold = must have fewer than 20% missing values
# drop NL, ME, IT, MK, BE, CY, IE, and HU
solar_semi_final.isnull().sum().sort_values(ascending=False)/solar_final.shape[0]#.mean()

region
NL    0.997238
ME    0.825663
IT    0.635276
MK    0.504197
BE    0.366116
CY    0.287277
IE    0.260983
HU    0.206285
LT    0.192171
CH    0.180513
RO    0.177273
SI    0.174902
BG    0.174772
GR    0.173597
SE    0.172706
LV    0.171248
EE    0.168573
FI    0.168465
ES    0.168356
PT    0.162288
PL    0.015724
FR    0.015702
DK    0.014158
DE    0.000130
CZ    0.000065
GB    0.000000
AT    0.000000
dtype: float64

In [285]:
solar_final = solar_semi_final.drop(columns=['NL','ME','IT','MK','BE','CY','IE','HU'])

In [286]:
# fill missing values
solar_final.fillna(0.0, inplace=True)

#### Wind

Summing and dropping the country regions where necessary

- Dropping bidding and control regions of: IT, DE, DK, IE (includes northern Ireland which is UK not IE), NO, SE
- Summing and dropping regions: GB
  - sum['GB_GBN', 'GB_NIR', 'GB_UKM'] --> 'GB'

In [252]:
#wind_subset_time.isnull().sum(axis=0)

In [253]:
#select column names excluding the country name, only control and bidding zones
DE_cols = [col for col in wind_subset_time.columns if 'DE' in col][1:]
DK_cols = [col for col in wind_subset_time.columns if 'DK' in col][1:]
IT_cols = [col for col in wind_subset_time.columns if 'IT' in col][1:]
GB_cols = [col for col in wind_subset_time.columns if 'GB' in col]
IE_cols = [col for col in wind_subset_time.columns if 'IE' in col][1:]
NO_cols = [col for col in wind_subset_time.columns if 'NO' in col][1:]
SE_cols = [col for col in wind_subset_time.columns if 'SE' in col][1:]

In [254]:
# new GB column
wind_subset_time['GB'] = wind_subset_time[GB_cols].sum(axis=1)

/Users/elizastarr/anaconda3/envs/datamining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [282]:
wind_semi_final = wind_subset_time.drop(columns=DE_cols+DK_cols+IT_cols+GB_cols+IE_cols+NO_cols+SE_cols)

Drop countries that still have too many null values by finding a threshold

In [283]:
# Require that each country have this many non-NA values.
# threshold = must have fewer than 20% missing values
# drop NL, ME, IT, MK, BE, CY, IE, and HU
wind_semi_final.isnull().sum().sort_values(ascending=False)/wind_final.shape[0]#.mean()

region
NL    0.997292
ME    0.829080
IT    0.622828
MK    0.513913
BE    0.358942
CY    0.301243
IE    0.275464
HU    0.221838
LT    0.208000
CH    0.196571
RO    0.193394
SI    0.191070
BG    0.190942
GR    0.189791
SE    0.188917
LV    0.187488
EE    0.184865
FI    0.184759
ES    0.184652
PT    0.178703
PL    0.015416
FR    0.015395
DK    0.013881
DE    0.000128
CZ    0.000064
GB    0.000000
AT    0.000000
dtype: float64

In [285]:
wind_final = wind_semi_final.drop(columns=['NL','ME','IT','MK','BE','CY','IE','HU'])

In [286]:
# fill missing values
wind_final.fillna(0.0, inplace=True)

Save to file

In [103]:
wind_final.to_csv('wind_generation.csv')
solar_final.to_csv('solar_generation.csv')